<a href="https://colab.research.google.com/github/NatEFFIE/NatEFFIE/blob/main/AB_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Задание* 7. A/B тестирование.


Шаг 1. Изучите материалы лекционных и практических занятий по темам раздела 6.

Шаг 2. Выполните A/B тестирование по предложенному датасету (алгоритм выполнения представлен в практическом занятии по теме 6.1).

Шаг 3. Опубликуйте файл расширения ipynb на платформе Odin.


Кейс: a/b-тест для мобильного приложения.

Имеется мобильное приложение для магазина по продаже продуктов питания. Выдвинута гипотеза о том, что смена шрифтов улучшит качество обслуживания по количеству клиентов, совершающих каждое событие. Для проверки этой гипотезы принято решение провести A/B-тест.   

Договорились принять решение по результатам A/B-теста.
Пользователей разбили на 2 группы: 247 -- контрольная группа со старыми шрифтами (это другая группа, по сравнению с разобранной); 248 -- экспериментальная с новыми шрифтами.

Описание данных

|Признак|Комментарий|
|--:|:--|
|`EventName`|название события|
|`DeviceIDHash`|уникальный идентификатор пользователя|
|`EventTimestamp`|время события|
|`ExpId`|номер группы|

Инструкция по выполнению проекта

Шаг 1. Загрузите данные
- Знакомство с данными;

Шаг 2. Подготовьте данные
- Корректировка заголовков;
- Типы данных;
- Удаление дублей.

Шаг 3. EDA
- Cколько всего событий?
- Сколько всего пользователей в логе?
- Сколько в среднем событий приходится на пользователя?
- Период теста: максимальная и минимальная даты.

Шаг 4. Анализ воронки событий
- Распределение событий: какие события и в каком количестве.
- Сколько пользователей совершали каждое из этих событий?
- Постройте воронку событий: какая доля пользователей проходит на следующий шаг воронки. На каком шаге теряете больше всего пользователей? Какая доля пользователей доходит от первого события до оплаты?

Шаг 5. Анализ результатов эксперимента
- Сколько пользователей в каждой группе?
- Проверьте гипотезу о наличие значимых отличий по результатам теста.

**Загрузка данных и импорт библиотек**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import statsmodels.api as sm

In [ ]:
application = pd.read_csv('ab_test_home.csv')

 **Подготовка данных**

In [ ]:
application.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163822 entries, 0 to 163821
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       163822 non-null  object
 1   DeviceIDHash    163822 non-null  int64 
 2   EventTimestamp  163822 non-null  object
 3   ExpId           163822 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 5.0+ MB


In [ ]:
application.duplicated().sum()

290

*В таблице нет пропущенных значений, но есть дублирующие строки, который необходимо удалить.*

In [ ]:
drop_ = application[application.duplicated()]
application = application.drop(drop_.index, axis = 0)

In [ ]:
application.duplicated().sum()

0

*Корректируем заголовки.*

In [ ]:
application.columns = ['event', 'user_id', 'event_time', 'group']

In [ ]:
application.head(5)

,event,user_id,event_time,group
0,PaymentScreenSuccessful,3518123091307005509,2019-07-25 11:28:47,248
1,CartScreenAppear,3518123091307005509,2019-07-25 11:28:47,248
2,PaymentScreenSuccessful,6217807653094995999,2019-07-25 11:48:42,248
3,CartScreenAppear,6217807653094995999,2019-07-25 11:48:43,248
4,MainScreenAppear,1850981295691852772,2019-07-25 20:31:42,247


*Приводим дату к формату datetime, выводим отдельным столбцом дни для подсчета длительности эксперимента.*

In [ ]:
application['event_time'] = pd.to_datetime(application['event_time'])

In [ ]:
application['date'] = application['event_time'].dt.date

*Посмотрим выборочно информацию на одного пользователя (устройство) - как видно из таблицы, пользователь может начать путь в приложении сразу с корзины (минуя стартовую страницу и страницы с продукцией). Видимо, продукция была помещена в корзину заранее - получается: первый вход пользователя с начала эксперимента может нести неполную информацию и искажать данные.*

In [ ]:
application_id = application[(application.user_id == 3518123091307005509)]
application_id

,event,user_id,event_time,group,date
0,PaymentScreenSuccessful,3518123091307005509,2019-07-25 11:28:47,248,2019-07-25
1,CartScreenAppear,3518123091307005509,2019-07-25 11:28:47,248,2019-07-25
9371,MainScreenAppear,3518123091307005509,2019-08-01 10:03:22,248,2019-08-01
9372,CartScreenAppear,3518123091307005509,2019-08-01 10:03:26,248,2019-08-01
9373,PaymentScreenSuccessful,3518123091307005509,2019-08-01 10:03:26,248,2019-08-01
9374,MainScreenAppear,3518123091307005509,2019-08-01 10:03:26,248,2019-08-01
9377,OffersScreenAppear,3518123091307005509,2019-08-01 10:03:36,248,2019-08-01
9383,CartScreenAppear,3518123091307005509,2019-08-01 10:03:44,248,2019-08-01
9386,PaymentScreenSuccessful,3518123091307005509,2019-08-01 10:03:49,248,2019-08-01
9389,CartScreenAppear,3518123091307005509,2019-08-01 10:03:50,248,2019-08-01


**Исследовательский анализ данных**

*Количество событий и пользователей*

In [ ]:
application['event'].count()

163532

In [ ]:
application['user_id'].nunique()

5062

In [ ]:
mean_event = application['event'].count() / application['user_id'].nunique()
round(mean_event,1)

32.3

*Итого:*

*   *событий - 163 532;*
*   *пользователей - 5 062;*
*   *среднее количество событий на пользователя - 32.3*


In [ ]:
application.groupby('group')['date'].value_counts().sort_index()

group  date      
247    2019-07-25        1
       2019-07-26        8
       2019-07-27       23
       2019-07-28       36
       2019-07-29       58
       2019-07-30      138
       2019-07-31      664
       2019-08-01    12306
       2019-08-02    10990
       2019-08-03    11024
       2019-08-04     9942
       2019-08-05    10949
       2019-08-06    11720
       2019-08-07    10091
248    2019-07-25        4
       2019-07-26        9
       2019-07-27        8
       2019-07-28       36
       2019-07-29       71
       2019-07-30      145
       2019-07-31      746
       2019-08-01    12274
       2019-08-02    13618
       2019-08-03    11683
       2019-08-04    11512
       2019-08-05    12741
       2019-08-06    12342
       2019-08-07    10393
Name: date, dtype: int64

In [ ]:
application['date'].nunique()

14

*Эксперимент длился с 25.07.19 по 7.08.19.*
*Итого 14 дней.*

**Анализ воронки событий**

*Посчитаем общее количество событий и количество событий, которое выполнили пользователи из тестовой и контрольной групп.*

In [ ]:
application['event'].value_counts()

MainScreenAppear           80852
OffersScreenAppear         31904
CartScreenAppear           27870
PaymentScreenSuccessful    22206
Tutorial                     700
Name: event, dtype: int64

In [ ]:
application.groupby('group')['event'].value_counts()

group  event                  
247    MainScreenAppear           39677
       OffersScreenAppear         15341
       CartScreenAppear           12548
       PaymentScreenSuccessful    10039
       Tutorial                     345
248    MainScreenAppear           41175
       OffersScreenAppear         16563
       CartScreenAppear           15322
       PaymentScreenSuccessful    12167
       Tutorial                     355
Name: event, dtype: int64

*Считаем долю пользователей по событиям.*

*Тут вопрос: как считать корректно. По факту, у нас есть общее количество посещений каждого шага - конверсию, на мой взгляд, логичнее рассчитывать из абсолютных величин, не ориентируясь на количество пользователей. Так как один пользователь мог 10 раз остановиться на первом шаге и всего лишь 1 раз дойти до последнего. Конверсия для него будет 0.1.*
*А другой - пройти весь путь в половине случаев и конверсия для него будет 0.5.*

*При этом, если мы рассчитаем просто количество пользователей, прошедших каждый степ, то оба попадут в группу, дошедшую до последнего события.*

In [ ]:
app_pivot = pd.pivot_table(application, index = 'group', columns = 'event', values = 'user_id', aggfunc = len)

In [ ]:
app_pivot

event,CartScreenAppear,MainScreenAppear,OffersScreenAppear,PaymentScreenSuccessful,Tutorial
group,,,,,
247,12548,39677,15341,10039,345
248,15322,41175,16563,12167,355


In [ ]:
app_pivot_1 = pd.pivot_table(application, index = 'group', columns = 'event', values = 'user_id', aggfunc = ['nunique'])


In [ ]:
app_pivot_1

nunique                                      \
event CartScreenAppear MainScreenAppear OffersScreenAppear   
group                                                        
247               1240             2482               1530   
248               1239             2501               1538   

                                        
event PaymentScreenSuccessful Tutorial  
group                                   
247                      1160      286  
248                      1185      282

In [ ]:
app_pivot_us = pd.DataFrame({'group': [247, 248], 'MainScreenAppear': [2482, 2501], 'OffersScreenAppear': [1530, 1538], 'CartScreenAppear': [1240, 1239], 'PaymentScreenSuccessful': [1160, 1185], 'Tutorial': [286, 282]})

In [ ]:
app_pivot = app_pivot[['MainScreenAppear','OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful', 'Tutorial']]


In [ ]:
app_pivot

event,MainScreenAppear,OffersScreenAppear,CartScreenAppear,PaymentScreenSuccessful,Tutorial
group,,,,,
247,39677,15341,12548,10039,345
248,41175,16563,15322,12167,355


In [ ]:
app_pivot_us

,group,MainScreenAppear,OffersScreenAppear,CartScreenAppear,PaymentScreenSuccessful,Tutorial
0,247,2482,1530,1240,1160,286
1,248,2501,1538,1239,1185,282


*Итак мы имеем два свода с данными: по общему количеству проведенных событий и по количеству пользователей на событие. Посчитаем конверсию по каждому шагу и по первому-итоговому(оплата) шагу для обеих таблиц.*

In [ ]:
m_to_o_all = (app_pivot['OffersScreenAppear'] / app_pivot['MainScreenAppear']) * 100
m_to_o_us = (app_pivot_us['OffersScreenAppear'] / app_pivot_us['MainScreenAppear']) * 100
o_to_c_all = (app_pivot['CartScreenAppear'] / app_pivot['OffersScreenAppear']) * 100
o_to_c_us = (app_pivot_us['CartScreenAppear'] / app_pivot_us['OffersScreenAppear']) * 100
c_to_p_all = (app_pivot['PaymentScreenSuccessful'] / app_pivot['CartScreenAppear']) * 100
c_to_p_us = (app_pivot_us['PaymentScreenSuccessful'] / app_pivot_us['CartScreenAppear']) * 100
total_all = (app_pivot['PaymentScreenSuccessful'] / app_pivot['MainScreenAppear']) * 100
total_us = (app_pivot_us['PaymentScreenSuccessful'] / app_pivot_us['MainScreenAppear']) * 100

In [ ]:
print(f'''Доля перехода: ГЛАВНАЯ СТРАНИЦА - СТРАНИЦА С ТОВАРОМ: {round(m_to_o_all,2)};
Доля пользователей: ГЛАВНАЯ СТРАНИЦА - СТРАНИЦА С ТОВАРОМ:
{round(m_to_o_us, 2)};

Доля перехода: СТРАНИЦА С ТОВАРОМ - КОРЗИНА: {round(o_to_c_all,2)};
Доля пользователей: СТРАНИЦА С ТОВАРОМ - КОРЗИНА:
{round(o_to_c_us,2)};

Доля перехода: КОРЗИНА - ОПЛАТА: {round(c_to_p_all,2)};
Доля пользователей: КОРЗИНА - ОПЛАТА:
{round(c_to_p_us, 2)};

Доля перехода: ГЛАВНАЯ СТРАНИЦА - ОПЛАТА: {round(total_all, 2)};
Доля пользователей: ГЛАВНАЯ СТРАНИЦА - ОПЛАТА:
{round(total_us,2)}''')


Доля перехода: ГЛАВНАЯ СТРАНИЦА - СТРАНИЦА С ТОВАРОМ: group
247    38.66
248    40.23
dtype: float64;
Доля пользователей: ГЛАВНАЯ СТРАНИЦА - СТРАНИЦА С ТОВАРОМ: 
0    61.64
1    61.50
dtype: float64;

Доля перехода: СТРАНИЦА С ТОВАРОМ - КОРЗИНА: group
247    81.79
248    92.51
dtype: float64;
Доля пользователей: СТРАНИЦА С ТОВАРОМ - КОРЗИНА: 
0    81.05
1    80.56
dtype: float64;

Доля перехода: КОРЗИНА - ОПЛАТА: group
247    80.00
248    79.41
dtype: float64;
Доля пользователей: КОРЗИНА - ОПЛАТА: 
0    93.55
1    95.64
dtype: float64;

Доля перехода: ГЛАВНАЯ СТРАНИЦА - ОПЛАТА: group
247    25.30
248    29.55
dtype: float64;
Доля пользователей: ГЛАВНАЯ СТРАНИЦА - ОПЛАТА: 
0    46.74
1    47.38
dtype: float64


*На мой взгляд, корректнее использовать результаты по общим переходам (безотносительно пользователей).*
*Самая большая потеря происходит при переходе из главной страницы в страницу с товарами - составляет в среднем для двух групп более 60%.*
*Конверсия в оплату от главной страницы составляет 25.3% и 29.55% для контрольной и тестовой группы с разницей в 4%.*

**Анализ результатов эксперимента**

*Считаем общее количество пользователей в каждой группе.*

In [ ]:
application.groupby('group')['user_id'].nunique()

group
247    2520
248    2542
Name: user_id, dtype: int64

*Группы совпадают по количеству (погрешность в двадцать два человека при общей выборке в 2 500 незначительна). Эксперимент проводился в течение 2ух недель. Таким образом количество выборки и длительность эксперимента можно признать удовлетворительной.*

*Для сравнения пропорций будем использовать z - тест для пропорций.*
*Сравним сначала по общему переходу. Потом по переходу по пользователям.*

*Формулируем нулевую и альтернативную гипотезы.*

***Нулевая:*** *конверсия в оплату для тестовой и контрольной группы одинакова, изменение шрифта не повлияло на конверсию*.

***Альтернативная:*** *конверсия в оплату для тестовой и контрольной группы различается, изменение шрифта влияет на конверсию*.

***Альфа:*** *0.05*

In [ ]:
for_stat = pd.DataFrame({'group': ['control', 'test'], 'main': [39677, 41175], 'payment': [10039, 12167]})
for_stat

,group,main,payment
0,control,39677,10039
1,test,41175,12167


In [ ]:
z, p_value = sm.stats.proportions_ztest([for_stat['payment'][1], for_stat['payment'][0]], [for_stat['main'][1], for_stat['main'][0]])

In [ ]:
z, p_value

(13.527831541287892, 1.0713756684652275e-41)

*Посчитаем z-статистику для пропорций вручную.*

In [ ]:
p = 0.2955 - 0.253
var1 = 0.2955 * (1 - 0.2955)
var2 = 0.253 * (1 - 0.253)
ese = np.sqrt(var1/41175 + var2/ 39677)
z = p/ese
z

13.562838718220808

*Данные немного отличаются из-за округления долей.*

*В целом, уровень p-значимости значительно меньше установленного порогового уровня в 0.05, что позволяет нам сделать вывод о возможности **отклонения** нулевой гипотезы о том, что шрифт не повлиял на поведение пользователей.*

*Теперь посчитаем статистику для конверсии по уникальным пользователям.*

In [ ]:
us_stat = pd.DataFrame({'group': ['control', 'test'], 'main': [2520, 2542], 'payment': [1160, 1185]})
us_stat

,group,main,payment
0,control,2520,1160
1,test,2542,1185


In [ ]:
z_us, p_value_us = sm.stats.proportions_ztest([us_stat['payment'][1], us_stat['payment'][0]], [us_stat['main'][1], us_stat['main'][0]])
z_us, p_value_us

(0.4174037691236318, 0.676383098446545)

*Если ориентироваться только на уникальных пользователей, то статистически различимой значимости не выявлено, так как p_value = 0,68.*

***Вывод: я бы рассматривала только первый показатель, так как можно заметить явное различие между конверсиями в оплату от общего количества просмотров главной страницы. Соответственно, можно использовать введение нового шрифта для приложения.***